In [6]:
import numpy as np
import pandas as pd

In [7]:
X_train = pd.read_csv('X_train.csv',header=None)
y_train = pd.read_csv('y_train.csv',header=None)

In [8]:
X_test = pd.read_csv('X_test.csv',header=None)
y_test = pd.read_csv('y_test.csv',header=None)

In [9]:
embeddings = np.load('ggvec_embeddings.npy')

In [10]:
X_train_embeddings = [(embeddings[i]+embeddings[j]) for i,j in zip(X_train[0], X_train[1])]

In [11]:
X_train_embeddings = np.array(X_train_embeddings)

X_test_embeddings = np.load('X_test_embeddings.npy')

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [26]:
model = keras.Sequential(
    [
        keras.Input(shape=(32,)),
        layers.Dense(32, activation="relu"),
        layers.Dense(16, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ]
)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
model.fit(X_train_embeddings, y_train,shuffle=True,validation_split = 0.1, epochs=20, batch_size = 128)

Train on 34101116 samples, validate on 3789013 samples
Epoch 1/20
34101116/34101116 [==============================] - 594s 17us/sample - loss: 0.6219 - accuracy: 0.6110 - val_loss: 0.7179 - val_accuracy: 0.6200
Epoch 2/20
34101116/34101116 [==============================] - 588s 17us/sample - loss: 0.6153 - accuracy: 0.6147 - val_loss: 0.7068 - val_accuracy: 0.6496
Epoch 3/20
34101116/34101116 [==============================] - 587s 17us/sample - loss: 0.6134 - accuracy: 0.6149 - val_loss: 0.7303 - val_accuracy: 0.6030
Epoch 4/20
34101116/34101116 [==============================] - 588s 17us/sample - loss: 0.6126 - accuracy: 0.6151 - val_loss: 0.7066 - val_accuracy: 0.6775
Epoch 5/20
34101116/34101116 [==============================] - 586s 17us/sample - loss: 0.6122 - accuracy: 0.6156 - val_loss: 0.7072 - val_accuracy: 0.6567
Epoch 6/20
34101116/34101116 [==============================] - 587s 17us/sample - loss: 0.6118 - accuracy: 0.6159 - val_loss: 0.7371 - val_accuracy: 0.5822
Epo

In [29]:
model.save('model_1')

INFO:tensorflow:Assets written to: model_1\assets


In [38]:
model = keras.models.load_model('model_1')

In [ ]:
X_test_embeddings = [(embeddings[i]+embeddings[j]) for i,j in zip(X_test[0], X_test[1])]
X_test_embeddings = np.array(X_test_embeddings)

In [37]:
_, accuracy = model.evaluate(X_test_embeddings, y_test)
print('Accuracy: %.2f' % (accuracy*100))

9472534/9472534 [==============================] - 387s 41us/sample - loss: 0.6179 - accuracy: 0.6254
Accuracy: 62.54


In [28]:
test_public = pd.read_csv('../data/comp90051-2020-sem2-proj1/test-public.txt', delimiter = '\t')

In [29]:
test_public_embeddings = [(embeddings[i]+embeddings[j]) for i,j in zip(test_public['Source'], test_public['Sink'])]

In [30]:
test_public_embeddings = np.array(test_public_embeddings)

In [6]:
type(test_public)

pandas.core.frame.DataFrame

In [42]:
predictions = model.predict(test_public_embeddings)

In [43]:
predictions

array([[0.4759963 ],
       [0.73856753],
       [0.5170013 ],
       ...,
       [0.4224368 ],
       [0.64865834],
       [0.39944652]], dtype=float32)

In [44]:
sum(predictions)

array([972.95557], dtype=float32)

In [49]:
pd.DataFrame(predictions).to_csv('predictions/prediction_1.csv')

In [2]:
from xgboost import XGBClassifier

In [3]:
xgbc = XGBClassifier(tree_method='gpu_hist', gpu_id=0, verbosity = 2)

In [12]:
xgbc.fit(X_train_embeddings, y_train[0])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan,
              monotone_constraints='(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='gpu_hist', validate_parameters=1, verbosity=2)

In [16]:
fitted_xgbc = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
              importance_type='gain', interaction_constraints='',\
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,\
              min_child_weight=1,\
              monotone_constraints='(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)',\
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,\
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,\
              tree_method='gpu_hist', validate_parameters=1, verbosity=2)

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
scores = cross_val_score(fitted_xgbc, X_train_embeddings, y_train[0], cv=3)


In [ ]:
print("Mean cross-validation score: %.2f" % scores)

In [13]:
scores

array([0.62579106, 0.62412139, 0.62480167])

In [35]:
testPredictions = xgbc.predict_proba(test_public_embeddings)

In [36]:
testPredictions

array([[0.59035087, 0.40964916],
       [0.34390414, 0.65609586],
       [0.47158813, 0.52841187],
       ...,
       [0.7383452 , 0.2616548 ],
       [0.38864052, 0.6113595 ],
       [0.50401324, 0.49598676]], dtype=float32)

In [37]:
sum(testPredictions)

array([1066.6683,  933.3305], dtype=float32)